In [ ]:
get_ipython().system('pip install db-dtypes')

In [ ]:
import pandas as pd
import numpy as np
from google.cloud import bigquery

In [ ]:
BIGQUERY_PROJECT = 'ironhacks-data'
bigquery_client = bigquery.Client(project=BIGQUERY_PROJECT)

In [ ]:
query = """
SELECT *
FROM `ironhacks-data.ironhacks_competition.unemployment_data`
WHERE uu_id='e201385d37b5f6eea30f6d6d4106dc6f'
"""

In [ ]:
query_job = bigquery_client.query(query)
unemployment_data = query_job.to_dataframe()

In [ ]:
unemployment_data.shape

In [ ]:
unemployment_data.columns

In [ ]:
unemployment_data.drop(['uu_id', 'countyfips', 'tract',
       'tract_name', 'edu_8th_or_less', 'edu_grades_9_11',
       'edu_hs_grad_equiv', 'edu_post_hs', 'edu_unknown',
       'top_category_employer1', 'top_category_employer2',
       'top_category_employer3', 'gender_female', 'gender_male', 'gender_na',
       'race_amerindian', 'race_asian', 'race_black', 'race_noanswer',
       'race_hawaiiannative', 'race_other', 'race_white'], axis=1, inplace=True)

In [ ]:
unemployment_data.drop_duplicates(inplace=True)
unemployment_data.sort_values(['week_number'])

In [ ]:
def add_missing_weeks(df):
    # Fill in missing weeks by taking the ceil of the average of prev and next
    for week in range(1, 37):

In [ ]:
unemployment_data['year'] = unemployment_data['timeperiod'].apply(lambda x: str(x)[:4])
unemployment_data['month'] = unemployment_data['timeperiod'].apply(lambda x: str(x)[4:6])
unemployment_data['day'] = unemployment_data['timeperiod'].apply(lambda x: str(x)[6:])

In [ ]:
unemployment_data

In [ ]:
unemployment_data['ds'] = pd.DatetimeIndex(unemployment_data['year'] + '-' + unemployment_data['month'] + '-' + unemployment_data['day'])

In [ ]:
unemployment_data

In [ ]:
unemployment_data.drop(['timeperiod', 'year', 'month', 'day', 'week_number'], axis=1, inplace=True)
unemployment_data.columns = ['y', 'ds']

In [ ]:
unemployment_data

In [ ]:
unemployment_data.sort_values(['ds'], inplace=True)

In [ ]:
unemployment_data

In [ ]:
get_ipython().system('pip install prophet')

In [ ]:
from prophet import Prophet

In [ ]:
ud = unemployment_data

In [ ]:
threshold_date = pd.to_datetime('2022-05-14')
mask = ud['ds'] < threshold_date

In [ ]:
# Split the data and select `ds` and `y` columns.
ud_train = ud[mask][['ds', 'y']]
ud_test = ud[~mask][['ds', 'y']]

In [ ]:
ud_train

In [ ]:
m = Prophet(weekly_seasonality=False,
        daily_seasonality=False,
        interval_width=0.95, 
        mcmc_samples = 500)

m.add_seasonality(
        name='monthly', 
        period=30.5, 
        fourier_order=5
    )

In [ ]:
m.fit(ud_train)

In [ ]:
future = m.make_future_dataframe(periods=20, freq='W')

In [ ]:
forecast = m.predict(df=future)

In [ ]:
m.fit(ud_train, show_console=True)

In [ ]:
m = Prophet(weekly_seasonality=False,
        daily_seasonality=False,
        interval_width=0.95, 
        mcmc_samples = 500)

m.add_seasonality(
        name='monthly', 
        period=30.5, 
        fourier_order=5
    )

In [ ]:
m.fit(ud_train, show_console=True)

In [ ]:
future = m.make_future_dataframe(periods=20, freq='W')

In [ ]:
forecast = m.predict(df=future)